In [ ]:
#This is for the Movie Mayhem Project!

In [ ]:
#This is for loading the modules we want to use for the project.


In [ ]:
#This is for loading the CSV and parsing into a DF


In [ ]:
#This block is for Robertos Code

In [ ]:
#This block is for Tanners code

In [ ]:
#This block is for Davids code

In [ ]:
#This block is for Matts code